In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Reference Notebook - https://www.kaggle.com/ravishah1/cots-faster-rcnn-training-w-tf-2-0-od-api-0-474

In [ ]:
import contextlib2
import io
import IPython
import json
import numpy as np
import os
import pathlib
import pandas as pd
import sys
import tensorflow as tf
import time
import pandas as pd
import numpy as np
import argparse
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

### Install Object Detectin API

In [ ]:
!git clone https://github.com/tensorflow/models    
# Check out a certain commit to ensure that future changes in the TF ODT API codebase won't affect this notebook.
# !cd models && git checkout ac8d06519


In [ ]:
!cd models

In [ ]:
!pip install tensorflow-gpu==2.6.0

In [ ]:
%%bash
cd models/research

# Compile protos.
protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
# Note: I fixed the version of some dependencies to make it work on Kaggle notebook. In particular:
# * scipy==1.6.3 to avoid the missing GLIBCXX_3.4.26 error
# * tensorflow to 2.6.0 to make it compatible with the CUDA version preinstalled on Kaggle.
# When Kaggle notebook upgrade to TF 2.7, you can use the default setup.py script:
# cp object_detection/packages/tf2/setup.py .

wget https://storage.googleapis.com/odml-dataset/others/setup.py
pip install -q --user .

# Test if the Object Dectection API is working correctly
python object_detection/builders/model_builder_tf2_test.py

In [ ]:
from object_detection.utils import visualization_utils as viz_utils

In [ ]:
print(tf.__version__)
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
print(tf.debugging.set_log_device_placement(True))

### Stratified KFold and Create Dataset

In [ ]:
%%writefile cross_validation_setup.py
"""
Splits a dataframe to a specified cross_validation framework
"""

import pandas as pd
import numpy as np
import argparse
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

parser = argparse.ArgumentParser(
    description="Cross Validation Setup")
parser.add_argument("-in",
                    "--input_dir",
                    help="The filepath to the input csv.", type=str)
parser.add_argument("-out",
                    "--output_dir",
                    help="The filepath to the output csv.", type=str)
parser.add_argument("-type",
                    "--type",
                    help="The type of cross_validation to use.", type=str)
parser.add_argument("-on",
                    "--on",
                    help="The column to stratisfy on.", type=str)
parser.add_argument("-folds",
                    "--folds",
                    help="The number of folds to create.", type=int)
parser.add_argument("-hold",
                    "--holdout",
                    help="The fold to holdout.", type=int)

args = parser.parse_args()

def cross_validation():
    df = pd.read_csv(args.input_dir)
    df = df[df.annotations!='[]']
    df = df.reset_index(drop=True)
    
    if args.type=='kfold':
        kf = KFold(n_splits=args.folds, shuffle=True, random_state=42)
        for f, (t_, v_) in enumerate(kf.split(X=df)):
            df.loc[v_, 'fold'] = f
            
    elif args.type=='skfold' and args.on=='video_id':
        # uses skf to get even distrubution of data from different videos
        kf = StratifiedKFold(n_splits=args.folds, shuffle=True, random_state=42)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.video_id)): 
            df.loc[v_, 'fold'] = f
            
    elif args.type=='gkfold' and args.on=='sequence':
        kf = GroupKFold(n_splits=args.folds)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.video_id, groups=df.sequence)): 
            df.loc[v_, 'fold'] = f
            
    else:
        raise Exception('Not Implemented')
        
    df.to_csv(args.output_dir, index=False)
    
if __name__ == '__main__':
    cross_validation()

In [ ]:
%%writefile generate_tfrecords.py

import os
from os.path import exists
import glob
import pandas as pd
import io
import json
import xml.etree.ElementTree as ET
import contextlib2
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from object_detection.dataset_tools import tf_record_creation_util
from collections import namedtuple

# Initiate argument parser
parser = argparse.ArgumentParser(
    description="TensorFlow TFRecord Generator")
parser.add_argument("-c",
                    "--csv_path",
                    help="Path to the train.csv file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored.", type=str)
parser.add_argument("-t",
                    "--train",
                    help="True if this is a training dataset, false if it is a validation dataset.", type=str)
parser.add_argument("-s",
                    "--shards",
                    help="The number of shards for the dataset", type=int)
parser.add_argument("-f",
                    "--holdout_fold",
                    help="The fold to holdout.", type=int)

args = parser.parse_args()

def create_tf_example(data_df: pd.DataFrame, video_id: int, video_frame: int):
    """
    Create a tf.Example entry for a given training image.
    """
    full_path = os.path.join(args.image_dir, os.path.join(f'video_{video_id}', f'{video_frame}.jpg'))
    with tf.io.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')

    height = image.size[1] # Image height
    width = image.size[0] # Image width
    filename = f'{video_id}:{video_frame}'.encode('utf8') # Unique id of the image.
    encoded_image_data = None # Encoded image bytes
    image_format = 'jpeg'.encode('utf8') # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box# (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    rows = data_df[(data_df.video_id == video_id) & (data_df.video_frame == video_frame)]
    for _, row in rows.iterrows():
        annotations = json.loads(row.annotations.replace("'", '"'))
        for annotation in annotations:
            xmins.append(annotation['x'] / width) 
            xmaxs.append((annotation['x'] + annotation['width']) / width) 
            ymins.append(annotation['y'] / height) 
            ymaxs.append((annotation['y'] + annotation['height']) / height) 

            classes_text.append('COTS'.encode('utf8'))
            classes.append(1)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example

def create_labels_file():
    label_map_str = """
    item {
        id: 1
        name: 'COTS'
        }
                    """

    if exists('dataset/label_map.pbtxt') is False:
        with open('dataset/label_map.pbtxt', 'w') as f:
            f.write(label_map_str)
        print('Successfully created label_map.pbtxt file')

if __name__ == '__main__':

    # label file
    create_labels_file()
    #writer = tf.python_io.TFRecordWriter(args.output_path)
    
    # setup df
    data_df = pd.read_csv(args.csv_path)
    if args.train =='train':
        data_df = data_df[data_df.fold != args.holdout_fold].reset_index(drop=True)
    else:
        data_df = data_df[data_df.fold == args.holdout_fold].reset_index(drop=True)
    
    # make records
    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
            tf_record_close_stack, args.output_path, args.shards)
        
        for index, row in data_df.iterrows():
            tf_example = create_tf_example(data_df, row.video_id, row.video_frame)
            output_shard_index = index % args.shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
    #writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))

In [ ]:
# %%bash
# !mkdir dataset
os.mkdir('dataset')

In [ ]:
# %%bash

# python cross_validation_setup.py \
#     -in ../input/tensorflow-great-barrier-reef/train.csv \
#     -out train.csv \
#     -type skfold \
#     -on video_id \
#     -folds 10 \
#     -hold 0

# mkdir dataset
from os.path import exists

def create_labels_file():
    label_map_str = """
    item {
        id: 1
        name: 'COTS'
        }
                    """
    if exists('dataset/label_map.pbtxt') is False:
        with open('dataset/label_map.pbtxt', 'w') as f:
            f.write(label_map_str)
        print('Successfully created label_map.pbtxt file')

create_labels_file()

# python generate_tfrecords.py \
#     -c train.csv \
#     -o dataset/cots_train \
#     -i ../input/tensorflow-great-barrier-reef/train_images \
#     -t train \
#     -s 4 \
#     -f 0

# python generate_tfrecords.py \
#     -c train.csv \
#     -o dataset/cots_val \
#     -i ../input/tensorflow-great-barrier-reef/train_images \
#     -t valid \
#     -s 4 \
#     -f 0

### Download Pretrained Model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xvzf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!rm ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
!tar -xvzf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
!rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

### Edit Config

In [ ]:
# # Faster R-CNN with Resnet-50 (v1), configuration for MSCOCO Dataset.
# # Users should configure the fine_tune_checkpoint field in the train config as
# # well as the label_map_path and input_path fields in the train_input_reader and
# # eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# # should be configured.

# model {
#   faster_rcnn {
#     num_classes: 90
#     image_resizer {
#       keep_aspect_ratio_resizer {
#         min_dimension: 600
#         max_dimension: 1024
#       }
#     }
#     feature_extractor {
#       type: 'faster_rcnn_resnet50'
#       first_stage_features_stride: 16
#     }
#     first_stage_anchor_generator {
#       grid_anchor_generator {
#         scales: [0.25, 0.5, 1.0, 2.0]
#         aspect_ratios: [0.5, 1.0, 2.0]
#         height_stride: 16
#         width_stride: 16
#       }
#     }
#     first_stage_box_predictor_conv_hyperparams {
#       op: CONV
#       regularizer {
#         l2_regularizer {
#           weight: 0.0
#         }
#       }
#       initializer {
#         truncated_normal_initializer {
#           stddev: 0.01
#         }
#       }
#     }
#     first_stage_nms_score_threshold: 0.0
#     first_stage_nms_iou_threshold: 0.7
#     first_stage_max_proposals: 300
#     first_stage_localization_loss_weight: 2.0
#     first_stage_objectness_loss_weight: 1.0
#     initial_crop_size: 14
#     maxpool_kernel_size: 2
#     maxpool_stride: 2
#     second_stage_box_predictor {
#       mask_rcnn_box_predictor {
#         use_dropout: false
#         dropout_keep_probability: 1.0
#         fc_hyperparams {
#           op: FC
#           regularizer {
#             l2_regularizer {
#               weight: 0.0
#             }
#           }
#           initializer {
#             variance_scaling_initializer {
#               factor: 1.0
#               uniform: true
#               mode: FAN_AVG
#             }
#           }
#         }
#       }
#     }
#     second_stage_post_processing {
#       batch_non_max_suppression {
#         score_threshold: 0.0
#         iou_threshold: 0.6
#         max_detections_per_class: 100
#         max_total_detections: 300
#       }
#       score_converter: SOFTMAX
#     }
#     second_stage_localization_loss_weight: 2.0
#     second_stage_classification_loss_weight: 1.0
#   }
# }

# train_config: {
#   batch_size: 1
#   optimizer {
#     momentum_optimizer: {
#       learning_rate: {
#         manual_step_learning_rate {
#           initial_learning_rate: 0.0003
#           schedule {
#             step: 0
#             learning_rate: .0003
#           }
#           schedule {
#             step: 900000
#             learning_rate: .00003
#           }
#           schedule {
#             step: 1200000
#             learning_rate: .000003
#           }
#         }
#       }
#       momentum_optimizer_value: 0.9
#     }
#     use_moving_average: false
#   }
#   gradient_clipping_by_norm: 10.0
#   fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED/model.ckpt"
#   from_detection_checkpoint: true
#   # Note: The below line limits the training process to 200K steps, which we
#   # empirically found to be sufficient enough to train the pets dataset. This
#   # effectively bypasses the learning rate schedule (the learning rate will
#   # never decay). Remove the below line to train indefinitely.
#   num_steps: 200000
#   data_augmentation_options {
#     random_horizontal_flip {
#     }
#   }
# }

# train_input_reader: {
#   tf_record_input_reader {
#     input_path: "PATH_TO_BE_CONFIGURED/mscoco_train.record"
#   }
#   label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
# }

# eval_config: {
#   num_examples: 8000
#   # Note: The below line limits the evaluation process to 10 evaluations.
#   # Remove the below line to evaluate indefinitely.
#   max_evals: 10
# }

# eval_input_reader: {
#   tf_record_input_reader {
#     input_path: "PATH_TO_BE_CONFIGURED/mscoco_val.record"
#   }
#   label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
#   shuffle: false
#   num_readers: 1
# }

In [ ]:
from string import Template

config_file_template = """
# SSD with Resnet 50 v1 FPN feature extractor, shared box predictor and focal
# loss (a.k.a Retinanet).
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 38.3 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: [1.0, 2.0, 0.5]
        scales_per_octave: 2
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        depth: 256
        class_prediction_bias_init: -4.6
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.0004
            }
          }
          initializer {
            random_normal_initializer {
              stddev: 0.01
              mean: 0.0
            }
          }
          batch_norm {
            scale: true,
            decay: 0.997,
            epsilon: 0.001,
          }
        }
        num_layers_before_predictor: 4
        kernel_size: 3
      }
    }
    feature_extractor {
      type: 'ssd_resnet50_v1_fpn_keras'
      fpn {
        min_level: 3
        max_level: 7
      }
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.0004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          scale: true,
          decay: 0.997,
          epsilon: 0.001,
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    loss {
      classification_loss {
        weighted_sigmoid_focal {
          alpha: 0.25
          gamma: 2.0
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    normalize_loc_loss_by_codesize: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  batch_size: 64
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 8
  use_bfloat16: true
  num_steps: $training_steps
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 5e-3
          total_steps: 2500
          warmup_learning_rate: 5e-4
          warmup_steps: $warmup_steps
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
}
train_input_reader: {
  label_map_path: "dataset/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "../input/cotsdataset/dataset/cots_train-?????-of-00004"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 2;
}

eval_input_reader: {
  label_map_path: "dataset/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "../input/cotsdataset/dataset/cots_val-?????-of-00004"
  }
}


"""

In [ ]:
# Define the training pipeline

TRAINING_STEPS = 100
WARMUP_STEPS = 20
PIPELINE_CONFIG_PATH='dataset/pipeline.config'

pipeline = Template(config_file_template).substitute(
    training_steps=TRAINING_STEPS, warmup_steps=WARMUP_STEPS)

with open(PIPELINE_CONFIG_PATH, 'w') as f:
    f.write(pipeline)

In [ ]:
MODEL_DIR='./cots_faster_rcnn_resnet50'
os.mkdir(MODEL_DIR)
# !mkdir {MODEL_DIR}

In [ ]:
# pip install tensorflow-gpu==1.15

### Train

In [ ]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr

In [ ]:
# # !python models/research/object_detection/model_main_tf2.py \
# #     --pipeline_config_path={PIPELINE_CONFIG_PATH} \
# #     --model_dir={MODEL_DIR} \
# #     --checkpoint_dir={MODEL_DIR} \
# #     --eval_timeout=0 \
# #     --alsologtostderr
# for i in os.listdir():
#     path = i
#     isdir = os.path.isdir(path) 
#     if isdir: 
#         print(i," : ", os.listdir(i))
# file1='faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
# file2='cots_faster_rcnn_resnet50'

### Export

In [ ]:
!python models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path={'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config'} \
    --trained_checkpoint_dir={'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint'} \
    --output_directory={MODEL_DIR}/output

In [ ]:
# !mkdir {MODEL_DIR}/output
# os.mkdir(MODEL_DIR+"/output")
# os.mkdir('./fine_tuned_model')

In [ ]:
# output_directory = './fine_tuned_model'

# # lst = os.listdir('../input/output/output')
# # lst = [l for l in lst if 'ckpt' in l and '.meta' in l]
# # steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
# # last_model = lst[steps.argmax()].replace('.meta', '')

# # last_model_path = os.path.join(model_dir, last_model)
# # print(last_model_path)
# !python models/research/object_detection/export_inference_graph.py \
#     --input_type=image_tensor \
#     --pipeline_config_path={'../input/output/output/pipeline.config'} \
#     --output_directory={output_directory} \
#     --trained_checkpoint_prefix={'../input/output/output'}

In [ ]:
model=tf.keras.models.load_model("cots_faster_rcnn_resnet50/output/saved_model")
os.listdir('cots_faster_rcnn_resnet101')


In [ ]:
# import matplotlib.pyplot as plt
# BASE_DIR = '/kaggle/input/tensorflow-great-barrier-reef'
# training_data = pd.read_csv(f'{BASE_DIR}/train.csv')
# testing_data = pd.read_csv(f'{BASE_DIR}/test.csv')
# training_data.info()

In [ ]:
# detect_fn1 = tf.saved_model.load('../input/output/output')
# detect_fn2 = tf.saved_model.load('cots_faster_rcnn_resnet50/output/saved_model')

In [ ]:
# testing_data.iloc[0]

In [ ]:
# xy=[]
# example = testing_data.iloc[9332, :]
# video_id = example['video_id']
# video_frame = example['video_frame']
# sequence = example['sequence']
# sequence_frame = example['sequence_frame']
# image_id = example['image_id']
# annotations = eval(example['annotations'])
# print(annotations)
# print(f'Image ID: {image_id}')
# for annotations in annotations:
#     xy.append([annotations['x'], annotations['y'], annotations['x'] + annotations['width'], annotations['y'] + annotations['height']])
# with Image.open(f'{BASE_DIR}/train_images/video_{video_id}/{video_frame}.jpg') as im:
#     for xy in xy:
#         draw = ImageDraw.Draw(im)
#         draw.rectangle(xy, outline='red', width=2)
#         im.save('fig1.jpg')
# Image.open('fig1.jpg')
# # print(video_frame)

In [ ]:
# import torchvision as tv
# print(f'Number of images: {len(training_data)}')
# with Image.open(f'{BASE_DIR}/train_images/video_{video_id}/{video_frame}.jpg') as im:
#     transform = tv.transforms.PILToTensor()
#     tensor = transform(im.copy())
# print(f'Channels: {tensor.shape[0]}\nHeight: {tensor.shape[1]}\nWidth: {tensor.shape[2]}')

# def load_image_into_numpy_array(image):
#     (im_width, im_height) = image.size
#     return np.array(image.getdata()).reshape(
#         (im_height, im_width, 3)).astype(np.uint8)

# image_path='../input/tensorflow-great-barrier-reef/train_images/video_0/1910.jpg'
# image = Image.open(image_path)
# # the array based representation of the image will be used later in order to prepare the
# # result image with boxes and labels on it.
# image_np = load_image_into_numpy_array(image)
# # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
# image_np_expanded = np.expand_dims(image_np, axis=0)
# # image_np_expanded
# plt.imshow(image)

pred = predict(image_np)
predictions=format_output(image_np,pred)
prediction_str = ' '.join(predictions)

# detect_fn(image_np_expanded)

In [ ]:
# import matplotlib.pyplot as plt
# from object_detection.utils import visualization_utils as viz_utils
# detections1 = detect_fn1(image_np_expanded)
# detections2 = detect_fn2(image_np_expanded)

# print(detections)
# category_index = {
#     1: {'id': 1, 'name': 'COTS'}}
# # end_time = time.time()
# #   elapsed.append(end_time - start_time)
# image_np = load_image_into_numpy_array(image)
# plt.rcParams['figure.figsize'] = [42, 21]
# label_id_offset = 1
# image_np_with_detections1 = image_np.copy()
# viz_utils.visualize_boxes_and_labels_on_image_array(
#         image_np_with_detections1,
#         detections1['detection_boxes'][0].numpy(),
#         detections1['detection_classes'][0].numpy().astype(np.int32),
#         detections1['detection_scores'][0].numpy(),
#         category_index,
#         use_normalized_coordinates=True,
#         max_boxes_to_draw=200,
#         min_score_thresh=.40,
#         agnostic_mode=False)
# plt.subplot(2, 1, 1)
# plt.imshow(image_np_with_detections1)

In [ ]:
# # %matplotlib inline
# image_np_with_detections2 = image_np.copy()
# viz_utils.visualize_boxes_and_labels_on_image_array(
#         image_np_with_detections2,
#         detections2['detection_boxes'][0].numpy(),
#         detections2['detection_classes'][0].numpy().astype(np.int32),
#         detections2['detection_scores'][0].numpy(),
#         category_index,
#         use_normalized_coordinates=True,
#         max_boxes_to_draw=200,
#         min_score_thresh=.40,
#         agnostic_mode=False)
# plt.subplot(2, 1, 1)
# plt.imshow(image_np_with_detections2)
# # plt.imshow(image_np_with_detections)


# # mean_elapsed = sum(elapsed) / float(len(elapsed))

In [ ]:
# Predict function
detect_fn1 = tf.saved_model.load('../input/output/output')

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# 
def predict(image):
#     image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image, axis=0)
    detections1 = detect_fn1(image_np_expanded)
    return detections1
    

In [ ]:
#visualise
%matplotlib inline

def show_pred(detections,image):
    category_index = {
    1: {'id': 1, 'name': 'COTS'}}
    image_np = load_image_into_numpy_array(image)
    plt.rcParams['figure.figsize'] = [42, 21]
    label_id_offset = 1
    image_np_with_detections1 = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections1,
            detections['detection_boxes'][0].numpy(),
            detections['detection_classes'][0].numpy().astype(np.int32),
            detections['detection_scores'][0].numpy(),
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.25,
            agnostic_mode=False)
    plt.subplot(2, 1, 1)
    plt.imshow(image_np_with_detections1)

In [ ]:
# show_pred(pred,image)

In [ ]:
# pred=predict(image)

In [ ]:
# for i in pred.keys():
#     print(i," : ",pred[i])
# pred['detection_boxes'][0][0]

### Clean up

In [ ]:
# !rm -rf dataset/
# !rm -rf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/
# !rm -rf models/

In [ ]:
# # Remove the dataset files to save space.
# !rm -rf dataset
# !rm -rf train_images
# !rm tensorflow-great-barrier-reef.zip

# # Remove other data downloaded during training.
# !rm -rf models
# !rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

<a href="./cots_faster_rcnn_resnet101/output/saved_model/keras_metadata.pb"> Download File </a>

In [ ]:
# print(os.listdir("./cots_faster_rcnn_resnet50/train/events.out.tfevents.1642756374.6caee2b3a66d.443.0.v2"),end="\n")

<a href="./cots_faster_rcnn_resnet50/output/checkpoint/checkpoint"> Download File </a>

In [ ]:
# %cd /kaggle/working
# from IPython.display import FileLink 
# FileLink(r'./cots_faster_rcnn_resnet101/output')

In [ ]:
# FileLink(r'./cots_faster_rcnn_resnet101/output')

In [ ]:
print("hi")

In [ ]:
# !python3 models/research/object_detection/export_saved_model.py \
#   --export_path=/tmp/movinet/ \
#   --model_id=a0 \
#   --causal=True \
#   --conv_type="3d" \
#   --num_classes=600 \
#   --use_positional_encoding=False \
#   --checkpoint_path=""

In [ ]:
# # os.listdir("models/research/object_detection/predictors")
# for i in os.listdir('dataset'):
#     s="<a href='./dataset/"+i+"'> Download File "+i+"</a>"
#     print(s,end="\n")


<a href='models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.config'> Download File cots_train-00000-of-00004</a>
<!-- <a href='./dataset/cots_train-00001-of-00004'> Download File cots_train-00001-of-00004</a>
<a href='./dataset/label_map.pbtxt'> Download File label_map.pbtxt</a>
<a href='./dataset/cots_train-00002-of-00004'> Download File cots_train-00002-of-00004</a>
<a href='./dataset/cots_val-00001-of-00004'> Download File cots_val-00001-of-00004</a>
<a href='./dataset/cots_val-00000-of-00004'> Download File cots_val-00000-of-00004</a>
<a href='./dataset/cots_val-00003-of-00004'> Download File cots_val-00003-of-00004</a>
<a href='./dataset/cots_train-00003-of-00004'> Download File cots_train-00003-of-00004</a>
<a href='./dataset/cots_val-00002-of-00004'> Download File cots_val-00002-of-00004</a> -->

In [ ]:
# import re
# import numpy as np

# output_directory = './fine_tuned_model'

# lst = os.listdir(MODEL_DIR)
# lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
# steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
# last_model = lst[steps.argmax()].replace('.meta', '')

# last_model_path = os.path.join(model_dir, last_model)
# print(last_model_path)
# !python /content/models/research/object_detection/export_inference_graph.py \
#     --input_type=image_tensor \
#     --pipeline_config_path={pipeline_fname} \
#     --output_directory={output_directory} \
#     --trained_checkpoint_prefix={last_model_path}

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

In [ ]:
# raw_detection_boxes
# detection_boxes
# detection_scores
# detection_anchor_indices
# num_detections
# detection_multiclass_scores
# detection_classes
# raw_detection_scores
# format_prediction(pred['detection_boxes'],pred['detection_scores'])

# Format output
def format_output(image,pred):
#     image_np = load_image_into_numpy_array(image)
    annot = []
    for i in range(300):
        if pred['detection_scores'][0][i].numpy()>0.25:
    #         print("Confidence :",pred['detection_scores'][0][i].numpy(),"Co-ordinates : ",pred['detection_boxes'][0][i].numpy())
            ls=pred['detection_boxes'][0][i].numpy()
            height,width,chal=image_np.shape
            xmin=ls[1]*width
            x2=ls[3]*width
            ymin=ls[0]*height
            y2=ls[2]*height
            height_img=x2-xmin
            width_img=y2-ymin
            arr={'x':xmin, 'y':ymin, 'width':width_img , 'height':height_img}
            annot.append('{:.2f} {} {} {} {}'.format(pred['detection_scores'][0][i], int(xmin), int(ymin), int(width_img), int(height_img)))

    print(annot)
    return annot

# pred['detection_scores'][0][i].numpy()

In [ ]:
# format_output(image,pred)
# image_np = load_image_into_numpy_array(image)
# height,width,chal=image_np.shape
# height,width

In [ ]:
type(iter_test) # iterate through all test set images
     
#     df_pred['annotations'] = predict(detections1, pixel_array)
#     env.predict(df_pred)

In [ ]:
# env = greatbarrierreef.make_env()# initialize the environment
# iter_test = env.iter_test()
# net = load_model(conf=0.25, iou=0.40)
# from tqdm.notebook import tqdm
# tqdm.pandas()

# annot = format_prediction(preds, img)
# pred_df=pd.dataFrame()
# pred_df['annotations'] = format_output(image,pred)
# env.predict()

In [ ]:
# for i in iter_test:
#     print(i)
#     break

In [ ]:
for (image_np, sample_prediction_df) in iter_test:
    pred = predict(image_np)
    predictions=format_output(image_np,pred)
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)
    print('Prediction:', prediction_str)

In [ ]:
# def find(image_np):
#     pred = predict(image_np)
#     anot=format_output(image_np,pred)
#     print(anot)
# find(image)

In [ ]:
image_path='../input/tensorflow-great-barrier-reef/train_images/video_0/1910.jpg'
image = Image.open(image_path)
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)
# image_np_expanded
# plt.imshow(image)
pred = predict(image_np)


# predictions=format_output(image_np,pred)
# prediction_str = ' '.join(predictions)

In [ ]:

show_pred(pred,image)

In [ ]:
print(prediction_str)

In [ ]:
output=format_output(image,pred)

In [ ]:
prediction_str = ' '.join(output)
prediction_str

In [ ]:
# Quiz